In [ ]:
import yaml
import torch
from pathlib import Path
from run_experiments import ExperimentManager
from utils.logging import setup_logging, get_logger
from utils.error_logging import get_error_logger

# Initialize logging
setup_logging()
logger = get_logger(__name__)
error_logger = get_error_logger()

# Create experiment manager
config_path = 'experiments/config.yaml'
manager = ExperimentManager(config_path)

# Log device information
device_info = manager._get_device_info()
logger.info(f"Running experiments on: {device_info}")
logger.info(f"Total experiments to run: {manager.total_experiments}")

# Run experiments
manager.run_experiments()